In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, sympify, lambdify
from ipywidgets import interact, FloatRangeSlider, IntSlider, Text, Dropdown, HBox, VBox

# 定义复合梯形法函数
def trapezoidal_rule(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    integral = h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])
    return x, y, integral

# 定义复合辛普森法函数
def simpson_rule(f, a, b, n):
    if n % 2 == 1:  # 确保n为偶数
        n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    integral = h / 3 * (y[0] + y[-1] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]))
    return x, y, integral

# 绘图函数
def plot_integral_approximation(func_str, interval, n, method):
    # 解析函数字符串
    x = symbols('x')
    expr = sympify(func_str)
    func = lambdify(x, expr, 'numpy')
    
    # 复合梯形法或复合辛普森法
    a, b = interval
    x_approx, y_approx, integral = (trapezoidal_rule if method == 'Trapezoidal' else simpson_rule)(func, a, b, n)
    
    # 为原函数生成足够密集的x值
    x_dense = np.linspace(a, b, 10000)
    y_dense = func(x_dense)
    
    # 绘图
    plt.figure(figsize=(10, 6))
    plt.plot(x_dense, y_dense, 'r-', label=f'f(x)')  # 原函数用红色实线表示
    
    if method == 'Simpson':
        # 绘制辛普森法的抛物线段
        for i in range(0, len(x_approx) - 2, 2):
             parabolas = np.array([x_approx[i], x_approx[i+1], x_approx[i+2]])
             parabolas_y = np.array([y_approx[i], y_approx[i+1], y_approx[i+2]])
             z = np.polyfit(parabolas, parabolas_y, 2)
             p = np.poly1d(z)
             plt.plot(np.linspace(x_approx[i], x_approx[i+2], 100), p(np.linspace(x_approx[i], x_approx[i+2], 100)), 'b--')
             plt.fill_between(np.linspace(x_approx[i], x_approx[i+2], 100), p(np.linspace(x_approx[i], x_approx[i+2], 100)), color='blue', alpha=0.3)
    else:
        # 逼近多项式用蓝色虚线表示
        plt.plot(x_approx, y_approx, 'b--', label=f'{method} Rule (n={n})')  
        plt.fill_between(x_approx, y_approx, color='blue', alpha=0.3)  # 填充颜色
    
    # 标注节点
    plt.scatter(x_approx, y_approx, color='green', zorder=5, label='Nodes')  # 使用绿色散点标注节点
    
    plt.title(f'Integral Approximation using {method} Rule\nIntegral ≈ {integral:.4f}')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()

# 创建交互式控件
func_str = Text(value='sin(x)', description='Function:')
interval = FloatRangeSlider(value=[0, np.pi], min=-100, max=100, step=0.1, description='Interval:')
method = Dropdown(options=['Trapezoidal', 'Simpson'], value='Trapezoidal', description='Method:')

# 动态调整节点数控件
def update_n_slider(method):
    if method == 'Trapezoidal':
        n_slider.max = 100
        n_slider.step = 1
    else:
        n_slider.max = 100
        n_slider.step = 2
        n_slider.value = 2 if n_slider.value % 2 != 0 else n_slider.value

n_slider = IntSlider(value=2, min=2, max=100, step=1, description='Nodes:')
method.observe(update_n_slider, names='value')

# 创建交互式界面
controls = VBox([func_str, interval, method, n_slider])
interact(plot_integral_approximation, func_str=func_str, interval=interval, n=n_slider, method=method)